<h1>Calibration Stereo Camera</h1>

<h2>Distortion causes by Camera</h2>

https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

Types of distortion:
*   Radial distortion: straight line --> curved. Radial distortion becomes larger the farther points are from the center of the image
*   Tangential (Decentering) distortion: causes by the image-taking lense is not aligned perfectly parallel with the imaging plane

We need to find 5 params known as **Distortion Coefficients** given by: `DC = (k1 k2 k3 p1 p2 )` (k1 k2 k3) is radial coeffs, (p1 p2) is tangential coeffs

Intrinsic params of the camera:
*   It is specifically assigned to a specific camera
*   Focal length $(f_x, f_y)$, optical centers $(c_x, c_y)$

Camera matrix
$$
\begin{bmatrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{bmatrix}
$$

Extrinsic params of the camera:
*   Rotation and Translation vectors to mapping a 3D point to a coordinate system

In [28]:
# At least 10 test patterns for camera calibration
''' 
Input:
    The 3D real world points (object points)
    The corresponding 2D points in the image (image points)
Output:
    The Extrinsic params
'''

import numpy as np
import cv2
import glob

In [29]:
chessSize = (9,6)
imSize = (640,480)

Find Chess Board Corners
*   Input: image, size of image, size of chessboard
*   Output: Chess board corners

In [30]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# object points
objp = np.zeros((chessSize[0]*chessSize[1],3),np.float32)
objp[:,:2] = np.mgrid[0:chessSize[0],0:chessSize[1]].T.reshape(-1,2)

# Store all the object points and image points from all the images
objpoints = [] # 3d point in real world space
imgpointsL = [] # 2d point in the image plane
imgpointsR = [] # 2d point in the image plane

imagesLeft = glob.glob('images/stereoLeft/*.png')
imagesRight = glob.glob('images/stereoRight/*.png')

for fLeft, fRight in zip(imagesLeft, imagesRight):
    imgLeft = cv2.imread(fLeft,0)
    imgRight = cv2.imread(fRight,0)

    # Find the chess board corners
    retL, corners = cv2.findChessboardCorners(imgLeft, chessSize, None)
    retR, corners = cv2.findChessboardCorners(imgRight, chessSize, None)

    # If found, add object points, image points (after refining them)
    if retL and retR == True:
        objpoints.append(objp)

        cornersL = cv2.cornerSubPix(imgLeft,corners, (11,11), (-1,-1), criteria)
        imgpointsL.append(cornersL)

        cornersR = cv2.cornerSubPix(imgRight,corners, (11,11), (-1,-1), criteria)
        imgpointsR.append(cornersR)

Calibration
*   Input: Chess board corners
*   Output: The camera matrix, DC, rotation & translation vectors

In [31]:
retL, mtxL, distL, rvecsL, tvecsL = cv2.calibrateCamera(objpoints, imgpointsL, imSize, None, None)
retR, mtxR, distR, rvecsR, tvecsR = cv2.calibrateCamera(objpoints, imgpointsR, imSize, None, None)

heightL, widthL = imgLeft.shape
heightR, widthR = imgRight.shape

newmtxL, roiL = cv2.getOptimalNewCameraMatrix(mtxL, distL, (widthL, heightL), 1, (widthL, heightL))
newmtxR, roiR = cv2.getOptimalNewCameraMatrix(mtxL, distL, (widthL, heightL), 1, (widthL, heightL))

Stereo Vision Undistortion

In [32]:
flags = 0
flags |= cv2.CALIB_FIX_INTRINSIC

# Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
# Hence intrinsic parameters are the same 

criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
retStereo, newmtxL, distL, newmtxR, distR, rot, trans, essentialMatrix, fundamentalMatrix = cv2.stereoCalibrate(objpoints, imgpointsL, imgpointsR, newmtxL, distL, newmtxR, distR, imgLeft.shape[::-1], criteria_stereo, flags)

Stereo Rectification

In [33]:
rectifyScale= 1
rectL, rectR, projMatrixL, projMatrixR, Q, roi_L, roi_R= cv2.stereoRectify(newmtxL, distL, newmtxR, distR, imgLeft.shape[::-1], rot, trans, rectifyScale,(0,0))

stereoMapL = cv2.initUndistortRectifyMap(newmtxL, distL, rectL, projMatrixL, imgLeft.shape[::-1], cv2.CV_16SC2)
stereoMapR = cv2.initUndistortRectifyMap(newmtxR, distR, rectR, projMatrixR, imgRight.shape[::-1], cv2.CV_16SC2)

print("Saving parameters!")
cv_file = cv2.FileStorage('stereoMap.xml', cv2.FILE_STORAGE_WRITE)

cv_file.write('stereoMapL_x',stereoMapL[0])
cv_file.write('stereoMapL_y',stereoMapL[1])
cv_file.write('stereoMapR_x',stereoMapR[0])
cv_file.write('stereoMapR_y',stereoMapR[1])

cv_file.release()

Saving parameters!
